In [1]:
import os
from pathlib import Path
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Correct project folder
dataset_path = r'C:\Users\chait\Downloads\FOOD_ESTI'   # <-- THIS IS YOUR REAL FOLDER

train_dir = os.path.join(dataset_path, 'train')
test_dir  = os.path.join(dataset_path, 'test')

print("Expecting train folder at:", train_dir)
print("Exists?:", Path(train_dir).exists())
print("Expecting test folder at:", test_dir)
print("Exists?:", Path(test_dir).exists())

if not Path(train_dir).exists() or not Path(test_dir).exists():
    raise FileNotFoundError(f"Check paths. train exists: {Path(train_dir).exists()}, test exists: {Path(test_dir).exists()}")

# transforms
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
}

train_dataset = datasets.ImageFolder(root=train_dir, transform=data_transforms['train'])
test_dataset  = datasets.ImageFolder(root=test_dir,  transform=data_transforms['test'])

BATCH_SIZE = 32
NUM_WORKERS = 0  # Windows notebook safe

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
test_loader  = DataLoader(test_dataset,  batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

print("Train samples:", len(train_dataset))
print("Test samples :", len(test_dataset))
print("Classes      :", train_dataset.classes)


Expecting train folder at: C:\Users\chait\Downloads\FOOD_ESTI\train
Exists?: True
Expecting test folder at: C:\Users\chait\Downloads\FOOD_ESTI\test
Exists?: True
Train samples: 75750
Test samples : 25250
Classes      : ['apple_pie', 'baby_back_ribs', 'baklava', 'beef_carpaccio', 'beef_tartare', 'beet_salad', 'beignets', 'bibimbap', 'bread_pudding', 'breakfast_burrito', 'bruschetta', 'caesar_salad', 'cannoli', 'caprese_salad', 'carrot_cake', 'ceviche', 'cheese_plate', 'cheesecake', 'chicken_curry', 'chicken_quesadilla', 'chicken_wings', 'chocolate_cake', 'chocolate_mousse', 'churros', 'clam_chowder', 'club_sandwich', 'crab_cakes', 'creme_brulee', 'croque_madame', 'cup_cakes', 'deviled_eggs', 'donuts', 'dumplings', 'edamame', 'eggs_benedict', 'escargots', 'falafel', 'filet_mignon', 'fish_and_chips', 'foie_gras', 'french_fries', 'french_onion_soup', 'french_toast', 'fried_calamari', 'fried_rice', 'frozen_yogurt', 'garlic_bread', 'gnocchi', 'greek_salad', 'grilled_cheese_sandwich', 'grille

In [2]:
import torch

def train(model, loader, criterion, optimizer, device):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

def validate(model, loader, criterion, device):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

In [3]:
import torch.nn as nn
import torch.optim as optim
from torchvision import models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
model = models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
num_classes = len(train_dataset.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)


Using device: cpu


c:\Users\chait\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\chait\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
EPOCHS = 10
best_val_acc = 0.0

for epoch in range(EPOCHS):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = validate(model, test_loader, criterion, device)
    
    print(f"Epoch {epoch+1}/{EPOCHS}")
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
    
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_food_model.pth")
        print("Best model saved!")

print(" Training complete!")


KeyboardInterrupt: 

In [3]:
import torch
from torchvision import transforms, models
from PIL import Image
import pandas as pd
import torch.nn as nn
import os
import pprint
import random
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
    [0.229, 0.224, 0.225])
])

MODEL_PATH = r"C:\Users\chait\OneDrive\Desktop\honeylu\best_food_model.pth"
nutrition_csv = r"C:\Users\chait\OneDrive\Desktop\honeylu\nutrition.csv"
test_image = "/mnt/data/apple.jpg"
state_dict = torch.load(MODEL_PATH, map_location=device)
trained_classes = state_dict["fc.weight"].shape[0]
print(f" Model was trained on {trained_classes} classes")
model = models.resnet18(weights=None)
model.fc = nn.Linear(model.fc.in_features, trained_classes)
model.load_state_dict(state_dict, strict=False)
print(" Pretrained model weights loaded")
your_classes = 0
try:
    dataset_dir = r"C:\Users\chait\OneDrive\Desktop\honeylu\dataset"
    if os.path.exists(dataset_dir):
        your_classes = len(os.listdir(dataset_dir))
        print(f" Detected {your_classes} food folders in dataset")
except:
    pass

if your_classes and your_classes != trained_classes:
    print(" Mismatch detected → Adjusting final layer")
    model.fc = nn.Linear(model.fc.in_features, your_classes)
model = model.to(device)
model.eval()
nutrition_df = pd.read_csv(nutrition_csv)
nutrition_df = nutrition_df.drop_duplicates(subset=['label'], keep='first')
nutrition_map = nutrition_df.set_index('label').to_dict('index')
class_names = list(nutrition_map.keys())
class VolumeEstimator:
    def estimate(self, image_path, food_class):
        volume_ml = random.uniform(100, 300)
        weight_g = random.uniform(80, 250)
        return volume_ml, weight_g

volume_estimator = VolumeEstimator()
def predict_with_nutrition(image_path):
    image = Image.open(image_path).convert('RGB')
    input_tensor = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(input_tensor)
        _, predicted_idx = torch.max(outputs, 1)

    predicted_class = class_names[predicted_idx.item() % len(class_names)]
    volume_ml, weight_g = volume_estimator.estimate(image_path, predicted_class)
    base_nutrition = nutrition_map.get(predicted_class, {})
    actual_nutrition = {
        k: v * (weight_g / 100) if isinstance(v, (int, float)) else v
        for k, v in base_nutrition.items()
    }

    return {
        'food': predicted_class,
        'volume_ml': round(volume_ml, 2),
        'weight_grams': round(weight_g, 2),
        'nutrition': actual_nutrition
    }
result = predict_with_nutrition(r"C:\Users\chait\OneDrive\Desktop\honeylu\apple.jpg")
print("\n Final Prediction Result:\n")
pprint.pprint(result)


Using device: cpu
 Model was trained on 101 classes
 Pretrained model weights loaded

 Final Prediction Result:

{'food': 'takoyaki',
 'nutrition': {'calories': 347.9860308116745,
               'carbohydrates': 43.49825385145931,
               'fats': 13.91944123246698,
               'fiber': 1.7399301540583725,
               'protein': 13.91944123246698,
               'sodium': 521.9790462175117,
               'sugars': 8.699650770291862,
               'weight': 173.99301540583724},
 'volume_ml': 118.18,
 'weight_grams': 173.99}
